In [19]:
## IMPORTS
import random
from deap import base


In [20]:
from deap import creator
from deap import tools
import numpy as np
import timeit
start = timeit.default_timer()
from stage import fullPrint
from datetime import datetime, date

In [21]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)


C:\Users\henry\miniconda3\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\henry\miniconda3\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [22]:
def generate_random_num_attr():
  original = np.array([0,0,0,0,0,0,0,0,0,1,0.1])
  noises = []
  sum_list = []

  noises.append(random.uniform(0,1))
  noises.append(random.uniform(0.1,2.5))
  noises.append(random.uniform(-0.1,0.4))
  noises.append(random.uniform(0.9,1.1))
  noises.append(random.uniform(1,1.4))
  noises.append(random.uniform(1, 1.3))
  noises.append(random.uniform(0.8, 1))
  noises.append(random.uniform(0.8, 1.1))
  noises.append(random.uniform(1.2, 1.5))
  noises.append(np.random.normal(0, 0.01))
  noises.append(np.random.normal(0, 0.05))

  for (item1, item2) in zip(original, noises):
      sum_list.append(item1+item2)
  
  return sum_list
noises = []
noises.append(np.random.normal(0, 0.025))
noises.append(np.random.normal(0, 0.075))
noises.append(np.random.normal(0, 0.015))
noises.append(np.random.normal(0, 0.005))
noises.append(np.random.normal(0, 0.0125))
noises.append(np.random.normal(0, 0.015))
noises.append(np.random.normal(0, 0.005))
noises.append(np.random.normal(0, 0.01))
noises.append(np.random.normal(0, 0.0125))
noises.append(np.random.normal(0, 0.0005))
noises.append(np.random.normal(0, 0.0025))
def mutate(individual):
  size = len(individual)
  for i in range(0,size):
    if random.random() > 0.9:
      individual[i] = individual[i] + noises[i]
  return individual,

In [23]:
toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("attr_bool", generate_random_num_attr)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, 1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [24]:
## change to our functinon
from stage import getStage
import numpy as np
import pandas as pd
import yahoo_fin.stock_info as yf
import random
from datetime import timedelta
SectorDict = {
    "Energy":"XLE","Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Staples":"XLP",
    "Health Care":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrial":"XLI",
    "Utilities":"XLU",
    "Materials":"XLB",
    "Financial":"XLF",
    "Real Estate":"XLRE"}
sectorDfList = []
for sector in SectorDict.values():
    sectorDfList.append(pd.read_pickle("stockData/sectorCharts/"+sector+".pkl"))
nyseDf = pd.read_html('stockList.html')
nyseDf = pd.DataFrame(nyseDf[0][0])
nyseList = []
for row in nyseDf.iterrows():
    ticker = str(row[1]).split()[1]
    if ticker == "Symbol":
        continue
    nyseList.append(ticker)
allList = []
nasdaqList = yf.tickers_nasdaq()
allList = set(nasdaqList + nyseList)

def evalOneMax(individual):
    now = datetime.now()
    now = now.strftime("%Y-%m-%d-%H:%M")
    print(individual[0])
    winningTrade = 0
    totalTrade = 0
    transaction = pd.read_pickle("transactionTemplate.pkl")
    transaction['holding'] = np.empty((len(transaction), 0)).tolist()
    goodSectorDf = sectorDfList[0].drop(['close','ticker','MA30','MA30Slope','RS'],axis=1)
    goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
    goodSectorDf.index = sectorDfList[0].index
    for date in sectorDfList[0].index:
        listOfSector = []
        for df in sectorDfList:
            try:
                if df.at[date,'MA30Slope'] > individual[0][9] and df.loc[date,'RS'] > individual[0][10]:
                    if df.at[date,'close'] > df.at[date,'MA30']:
                        listOfSector.append(df.at[date,'ticker'])
            except: 
                pass
        goodSectorDf.at[date,'Sectors'] = set(listOfSector)
    for symbol in random.sample(nyseList,1000):
        df = getStage(symbol,individual[0],goodSectorDf)
        inStage = False
        buyTwice = False
        if df.empty:
            continue
        buyPrice = 0
        secondBuyPrice = 0
        drift = -1
        for index, element in df.iterrows():
            if drift == -1:
                drift = index.weekday()
            index = index - timedelta(days= drift)
            index = index.strftime('%Y-%m-%d')
            if element.Stage == "Stage 2" or element.Stage == "Buy":
                if element.Stage == "Buy" and not inStage:
                    buyPrice = element.close
                transaction.loc[index]['holding'].append((symbol,element.close,0))
                if buyTwice:
                    transaction.loc[index]['holding'].append((symbol,element.close,0))
                if element.Stage == "Buy" and inStage:
                    secondBuyPrice = element.close
                    transaction.loc[index]['holding'].append((symbol,element.close,0))
                    buyTwice = True
                inStage = True
                continue
            if element.Stage == "Sell":
                transaction.loc[index]['holding'].append((symbol,element.close,-1))
                returnRate = element.close / buyPrice
                if returnRate > 1:
                    winningTrade += 1
                    totalTrade += 1
                else:
                    totalTrade += 1
                if buyTwice:
                    transaction.loc[index]['holding'].append((symbol,element.close,-1))
                    returnRate = element.close / secondBuyPrice
                    if returnRate > 1:
                        winningTrade += 1
                        totalTrade += 1
                    else:
                        totalTrade += 1
                buyPrice = 0
                secondBuyPrice = 0
                inStage = False
                buyTwice = False
    total = 100
    prevIndex = ''
    first = True
    transactionCopy = transaction
    transactionCopy['total'] = 0
    for index, element in transactionCopy.iterrows():
        index = index.strftime('%Y-%m-%d')
        if len(element.holding) == 0:
            transactionCopy['total'][index] = total
            continue
        if first:
            first = False
            transactionCopy['total'][index] = total
        else:
            prevData = transactionCopy.loc[prevIndex].holding
            if len(prevData):
                total = 0
                removeList = []
                close = 0
                for i in range(len(prevData)):
                    for j in range(len(element.holding)):
                        if element.holding[j][0] == prevData[i][0]:
                            close = element.holding[j][1]
                            if element.holding[j][2] == -1:
                                removeList.append(element.holding[j])
                            break
                    total += close*prevData[i][2]
                transactionCopy['total'][index] = total
                for delete in removeList:
                    if delete in element.holding:
                        element.holding.remove(delete)
            else:
                transactionCopy['total'][index] = transactionCopy.loc[prevIndex].total
        if len(element.holding):
            allocation = total/len(element.holding)
        for i in range(len(element.holding)):
            element.holding[i] = (element.holding[i][0],element.holding[i][1],allocation/element.holding[i][1])
        prevIndex = index
    stockHolding  = 0
    for i in transactionCopy.iterrows():
        stockHolding += len(i[1]['holding'])
    print(stockHolding/(transactionCopy.shape[0]))
    print(transactionCopy.iloc[-1]['total'])
    if totalTrade==0:
        print(winningTrade, totalTrade, 0)
    else:
        print(winningTrade, totalTrade, winningTrade/totalTrade)
    if stockHolding/(transactionCopy.shape[0])<8:
        return 0,
    resultFile = open("resultML1.txt","a")
    resultFile.write(str(individual[0])+"\n")
    resultFile.write(str(stockHolding/(transactionCopy.shape[0]))+"\n")
    resultFile.write(str(transactionCopy.iloc[-1]['total'])+"\n")
    resultFile.write(str(winningTrade)+" "+str(totalTrade)+" "+str(winningTrade/totalTrade)+"\n")
    resultFile.close()
    return transactionCopy.iloc[-1]['total'],

In [25]:
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", mutate)
toolbox.register("select", tools.selTournament, tournsize=3)

In [26]:
def main():
    pop = toolbox.population(n=512)
    
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
        if ind.fitness.values[0]<=100:
            del ind.fitness.values
    badInd = [ind for ind in pop if not ind.fitness.valid]
    while len(badInd)!=0:
        for ind in badInd:
            index = pop.index(ind)
            temp = toolbox.individual()
            pop[index] = temp
            del pop[index].fitness.values
        badInd = [ind for ind in pop if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, badInd)
        for ind, fit in zip(badInd, fitnesses):
            ind.fitness.values = fit
            if ind.fitness.values[0]<=100:
                del ind.fitness.values
        badInd = [ind for ind in pop if not ind.fitness.valid]
    
        
    # Begin the evolution
    for g in range(9):
        print("-- Generation %i --" % g)
        resultFile = open("resultML1.txt","a")
        resultFile.write("-- Generation %i --" % g+"\n")
        resultFile.close()
        
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.5:
                toolbox.mate(child1[0], child2[0])
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < 0.2:
                toolbox.mutate(mutant[0])
                del mutant.fitness.values        
    
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        # Replace population
        pop[:] = offspring
    
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)
        
        
    print("-- End of (successful) evolution --")
    
    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))


In [27]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
main()

[0.053008835981324665, 0.7360613578446238, 0.3207511068384562, 0.9876480593451332, 1.235080820111744, 1.1928069583156715, 0.9228375388671891, 0.8990425229317595, 1.2526199818723673, 0.992844476416354, 0.11002755584281063]
14.581148121899362
5597.646127024454
86 287 0.29965156794425085
[0.7985165894618824, 1.819693409873709, 0.23378765976340712, 1.0395187123746261, 1.3171137987307442, 1.2090735835002366, 0.9044795770300872, 1.0683498157318074, 1.4718233266226666, 1.013107740317196, 0.17903903475833075]
